In [1]:
!pip install -U -q transformers peft

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel
import torch

In [3]:
from huggingface_hub import login

login(token="<your hugging face token>")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
base_model_name = "google/gemma-2-2b-it"
adapter_path = "Researcher-024/gemma-2-2b-it-ft-adapter"

tokenizer = AutoTokenizer.from_pretrained(base_model_name, device_map = "auto")
base_model = AutoModelForCausalLM.from_pretrained(base_model_name, device_map = "auto")

model = PeftModel.from_pretrained(base_model, adapter_path)

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/166M [00:00<?, ?B/s]

In [7]:
def generate_prompt(input_text):
    instruction = "Extract all License Terms from the Statement given below in the following JSON format:\njson{ ... }"
    messages = f"<start_of_turn>Instruction {instruction} <end_of_turn>\n<start_of_turn>user {input_text} <end_of_turn>\n<start_of_turn>model"
    return messages

In [12]:
def generate_completion(input_text, max_length=1000, temperature=0.6, top_p=0.9):
    # Create the prompt using the generate_prompt function
    prompt = generate_prompt(input_text)

    input_ids = tokenizer(prompt, return_tensors="pt", add_special_tokens = True)
    outputs = model.generate(**input_ids, max_new_tokens=max_length, temperature = temperature, do_sample = True, pad_token_id = tokenizer.eos_token_id)
    #print(tokenizer.decode(outputs[0]))

    # Decode and return the generated text
    return tokenizer.decode(outputs[0], skip_special_tokens = True)


In [13]:
# Example usage
input_text = '''Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:The above copyright notice and this permission notice (including the next paragraph) shall be included in all copies or substantial portions of the Software. THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.'''
output = generate_completion(input_text)
print(output)

Instruction Extract all License Terms from the Statement given below in the following JSON format:
json{ ... } 
user Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:The above copyright notice and this permission notice (including the next paragraph) shall be included in all copies or substantial portions of the Software. THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHE